In [1]:
import sys
sys.path.insert(0, '../models')

from drn_pytorch import drn_c_42
from drn_pytorch import drn_d_105

import drn
import importlib
importlib.reload(drn)
make_drn = drn.make_drn

import chainer
import chainer.links as L
import chainer.functions as F
import numpy as np
import torch
import torch.onnx
from torch.autograd import Variable
from chainer import serializers
from sequential import Sequential

chainer.config.train = False
print(chainer.config.show())

chainer_model = make_drn(out_middle=True)
serializers.load_npz('../models/drn.npz', chainer_model)

pytorch_model = drn_d_105(pretrained=True, out_middle=True)
# pytorch_model = drn_c_42(pretrained=True, out_middle=True)
pytorch_model.eval()

/home/shunta/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/cupy-3.0.0a1-py3.6-linux-x86_64.egg/cupy/core/fusion.py:659: FutureWarning: cupy.core.fusion is experimental. The interface can change in the future.
  util.experimental('cupy.core.fusion')


cudnn_deterministic   False
debug                 False
enable_backprop       True
keep_graph_on_report  False
train                 False
type_check            True
use_cudnn             auto
use_cudnn_tensor_core auto
None


DRN(
  (layer0): Sequential(
    (0): Conv2d (3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
  )
  (layer1): Sequential(
    (0): Conv2d (16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
  )
  (layer2): Sequential(
    (0): Conv2d (16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
  )
  (layer3): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d (64, 256, kernel_

In [2]:
np_x = np.random.rand(1, 3, 224, 224).astype(np.float32)
torch_x = Variable(torch.from_numpy(np_x))

y_t, mids_t = pytorch_model(torch_x)
y_c, mids_c = chainer_model(np_x)

In [3]:
for t in mids_t:
    print(t.data.numpy().shape)
    
print('')

for c in mids_c:
    print(c.array.shape)

(1, 16, 224, 224)
(1, 16, 224, 224)
(1, 32, 112, 112)
(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 28, 28)
(1, 2048, 28, 28)
(1, 512, 28, 28)
(1, 512, 28, 28)

(1, 16, 224, 224)
(1, 16, 224, 224)
(1, 32, 112, 112)
(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 28, 28)
(1, 2048, 28, 28)
(1, 512, 28, 28)
(1, 512, 28, 28)


In [4]:
for i, (t, c) in enumerate(zip(mids_t, mids_c)):
    print(i, t.data.numpy().shape, c.array.shape, end='')
    try:
        a = t.data.numpy() != 0
        b = c.array != 0
        print(np.sum(a != b))
        np.testing.assert_allclose(t.data.numpy() != 0, c.array != 0)
    except Exception as e:
#         print(np.where(t.data.numpy() != c.array))
#         print(e)
        pass

0 (1, 16, 224, 224) (1, 16, 224, 224)0
1 (1, 16, 224, 224) (1, 16, 224, 224)5960
2 (1, 32, 112, 112) (1, 32, 112, 112)13038
3 (1, 256, 56, 56) (1, 256, 56, 56)45819
4 (1, 512, 28, 28) (1, 512, 28, 28)17744
5 (1, 1024, 28, 28) (1, 1024, 28, 28)88932
6 (1, 2048, 28, 28) (1, 2048, 28, 28)134821
7 (1, 512, 28, 28) (1, 512, 28, 28)42953
8 (1, 512, 28, 28) (1, 512, 28, 28)61296


In [5]:
pytorch_model.layer3[0]

Bottleneck(
  (conv1): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (downsample): Sequential(
    (0): Conv2d (32, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  )
)

In [6]:
for c in chainer_model.layer3[0].children():
    print(type(c))
    if isinstance(c, Sequential):
        for a in c:
            print(type(a))

<class 'chainer.links.connection.convolution_2d.Convolution2D'>
<class 'chainer.links.connection.convolution_2d.Convolution2D'>
<class 'chainer.links.normalization.batch_normalization.BatchNormalization'>
<class 'sequential.Sequential'>
<class 'chainer.links.connection.convolution_2d.Convolution2D'>
<class 'chainer.links.normalization.batch_normalization.BatchNormalization'>
<class 'chainer.links.connection.convolution_2d.Convolution2D'>
<class 'chainer.links.normalization.batch_normalization.BatchNormalization'>
<class 'chainer.links.normalization.batch_normalization.BatchNormalization'>
